In [15]:
from tokenizers import ByteLevelBPETokenizer, Tokenizer
import tempfile
import os
import json

In [16]:
def build_vocab_freqbased(V_PATH="./data/mwoz-bpe.tokenizer.json", recreate=False): # [ no of turns , src, tgt, act_vecs, hierarchial_act_vecs]
    if not recreate:
        if os.path.exists(V_PATH):
            print("Vocab Exists: ", V_PATH)
            # tokenizer = ByteLevelBPETokenizer()
            tokenizer = Tokenizer.from_file(V_PATH)
            return tokenizer
        
    split_name = 'train'
    file_path = 'hdsa_data/hdsa_data/'
    data_dir = 'data'
    dataset_file = open(file_path+split_name+'.json', 'r')
    dataset = json.load(dataset_file)

    idxtoword = {}
    wordtoidx ={}
    idxtoword[0]='PAD'
    idxtoword[1]='UNK'
    idxtoword[2] = 'SOS'
    idxtoword[3]='EOS'
    i = 4
    
    lines = []
    for x in dataset:
        dialog_file = x['file']
        src = []
        for turn_num, turn in enumerate(x['info']):
            user= turn['user'].lower().strip()
            sys = turn['sys'].lower().strip()
            lines.append(user)
            lines.append(sys)

    print(f"{len(lines)} lines in data.")
    # write to tmp
    fp = tempfile.NamedTemporaryFile("w", delete=False)
    for l in lines:
        fp.write(l+"\n")
    fp.close()
    print(fp.name)
    
    # build tokenizer
    tokenizer = ByteLevelBPETokenizer()
    tokenizer.train([fp.name], 
                    vocab_size=30000 , 
                    special_tokens=["PAD", "SOS", "EOS", "UNK"]
                   )
    tokenizer.save(V_PATH)
    
    # delete tmp file
    os.system(f"rm {fp.name}")
    return tokenizer


tokenizer = build_vocab_freqbased()
vocab_size = len(tokenizer.get_vocab())
print(vocab_size)

Vocab Exists:  ./data/mwoz-bpe.tokenizer.json
5694


In [17]:
# tokenizer.get_vocab()

In [18]:
inp = tokenizer.encode("there is [value_count] that arrives at [train_arriveby] . how many tickets do you need ? EOS")

In [19]:
print(inp.ids)

[489, 307, 265, 300, 66, 321, 64, 337, 707, 357, 265, 317, 66, 514, 64, 267, 618, 626, 730, 382, 274, 352, 283, 224, 2]


In [20]:
vocab_dict = tokenizer.get_vocab()
inv_vocab_dict = {v: k for (k, v) in vocab_dict.items()}
def convert_ids_to_tokens(id_list):
  tokens = []
  for ids in id_list:
    tokens.append(inv_vocab_dict[ids])
  return tokens

In [21]:
print(convert_ids_to_tokens(inp.ids))

['there', 'Ġis', 'Ġ[', 'value', '_', 'count', ']', 'Ġthat', 'Ġarrives', 'Ġat', 'Ġ[', 'train', '_', 'arriveby', ']', 'Ġ.', 'Ġhow', 'Ġmany', 'Ġtickets', 'Ġdo', 'Ġyou', 'Ġneed', 'Ġ?', 'Ġ', 'EOS']


In [22]:
tokenizer.decode(inp.ids)

'there is [value_count] that arrives at [train_arriveby] . how many tickets do you need ? '

2